<a href="https://colab.research.google.com/github/RMJ593/insight/blob/master/Insight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install tensorflow gtts pytesseract opencv-python
!apt-get install -y libportaudio2 tesseract-ocr tesseract-ocr-eng tesseract-ocr-hin tesseract-ocr-mal
!pip install langdetect

import tensorflow as tf
import numpy as np
import cv2
import pytesseract
from google.colab import files
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, decode_predictions, preprocess_input
from gtts import gTTS
from langdetect import detect
from IPython.display import Audio, display
import os


model = MobileNetV2(weights="imagenet")

def classify_image(image_path):
    """Loads an image and classifies it using MobileNetV2."""
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)

    predictions = model.predict(image)
    decoded_predictions = decode_predictions(predictions, top=1)[0]
    object_name = decoded_predictions[0][1]
    confidence = decoded_predictions[0][2]

    return object_name, confidence

def extract_text(image_path):
    """Extracts text from an image using OCR (supports English, Hindi, and Malayalam)."""
    image = cv2.imread(image_path)
    if image is None:
        return None, None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    text = pytesseract.image_to_string(gray, lang="eng+hin+mal").strip()

    if text:
        try:
            detected_lang = detect(text)
        except:
            detected_lang = "en"
    else:
        detected_lang = None

    return text if text else None, detected_lang


uploaded = files.upload()


for filename in uploaded.keys():
    detected_text, language = extract_text(filename)

    if detected_text:
        print(f"Extracted Text: {detected_text} (Language: {language})")

        lang_code = "en"
        if language == "hi":
            lang_code = "hi"
        elif language == "ml":
            lang_code = "ml"

        response_text = f"{detected_text}"
    else:
        object_name, confidence = classify_image(filename)
        print(f"Detected Object: {object_name} (Confidence: {confidence:.2f})")
        response_text = f"The object is {object_name}."
        lang_code = "en"


    tts = gTTS(response_text, lang=lang_code)
    audio_file = "response.mp3"
    tts.save(audio_file)


    display(Audio(audio_file, autoplay=True))


    # files.download(audio_file)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-hin is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-mal is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


Saving myntra.png to myntra.png
Extracted Text: WA

Myntra (Language: cy)
